<a href="https://colab.research.google.com/github/Nikhil-4-Pal/Generative-AI-4-ME/blob/main/Generative_Adversarial_Network_using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Adversarial Network

# Consist of Two Parts
* 1. Generator
* 2. Discriminator

# 1. vanilla Generative Adversarial Network

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [ ]:
class Discriminator(nn.Module):

  def __init__(self ,img_dims):
    super().__init__()
    self.disc = nn.Sequential(
        nn.Linear(img_dims , 128),
        nn.LeakyReLU(0.1),
        nn.Linear(128,64),
        nn.LeakyReLU(0.1),
        nn.Linear(64,32),
        nn.LeakyReLU(0.1),
        nn.Linear(32,16),
        nn.LeakyReLU(0.1),
        nn.Linear(16,1),
        nn.Sigmoid()
    )

  def forward(self , x):
    return self.disc(x)


# class Generator

class Generator(nn.Module):

  def __init__(self , z_dims ,img_dims ):
    super().__init__()

    self.gen = nn.Sequential(
        nn.Linear(z_dims , 256),
        nn.LeakyReLU(0.1),
        nn.Linear(256,128),
        nn.LeakyReLU(0.1),
        nn.Linear(128,64),
        nn.LeakyReLU(0.1),
        nn.Linear(64,img_dims),
        nn.Tanh()
    )

  def forward(self , x):
    return self.gen(x)


# Hyperparameters

device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dims = 64
img_dims = 28*28*1
batch_size = 32
num_epochs = 100


disc = Discriminator(img_dims).to(device)
gen = Generator(z_dims , img_dims).to(device)
fixed_noise = torch.randn((batch_size,z_dims)).to(device)
transforms = transforms.Compose(
    [transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))]
)

# loading the dataset

In [ ]:
dataset = datasets.MNIST(root="./data",transform=transforms,download=True)
loader = DataLoader(dataset,batch_size=batch_size , shuffle = True)
opt_disc = optim.Adam(disc.parameters() , lr = lr)
opt_gen = optim.Adam(gen.parameters(),lr =lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f'runs/GAN_MNIST/fake')
writer_real = SummaryWriter(f'runs/GAN_MNIST/real')
step = 0

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.60MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 65.5kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.09MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.83MB/s]


# Training Pipeline

In [ ]:
for epoch in range(num_epochs):
  for batch_idx , (real_img , _)  in enumerate(loader):
    real_img = real_img.view(-1 ,784).to(device)
    batch_size = real_img.shape[0]


    # training of the discriminator : here the goal is to maximize the loss log(D(real_img)) + log(1 - D(G(z)))

    noise = torch.randn((batch_size,z_dims)).to(device)
    fake_img = gen(noise)
    disc_real = disc(real_img).view(-1)
    lossD_real_img = criterion(disc_real , torch.ones_like(disc_real))
    disc_fake  = disc(fake_img).view(-1)
    lossD_fake_img = criterion(disc_fake , torch.zeros_like(disc_fake))
    lossD = (lossD_real_img + lossD_fake_img) / 2
    disc.zero_grad()
    lossD.backward(retain_graph = True)
    opt_disc.step()


    # now to train the generator : here the goal was to minimize the log(1- D(G(z)))  <--> maximizing the log(D(G(z)))

    output = disc(fake_img).view(-1)
    lossG = criterion(output , torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()


    # for tensorboard
    if batch_idx == 0 :
      print(
          f"epochs[{epoch}/{num_epochs}]"
          f"loss D :{lossD : .4f} , loss G : {lossG : .4f} "
      )

    with torch.no_grad():
      fake = gen(fixed_noise).reshape(-1,1,28,28)
      data = real_img.reshape(-1,1,28,28)
      img_grid_fake = torchvision.utils.make_grid(fake , normalize=True)
      img_grid_real = torchvision.utils.make_grid(data , normalize=True)

      writer_fake.add_image(
          "MNIST Fake Images", img_grid_fake , global_step = step
      )

      writer_real.add_image(
          "MNIST Real Images" , img_grid_real , global_step = True
      )

      step +=1

epochs[0/100]loss D : 0.6978 , loss G :  0.6259 
epochs[1/100]loss D : 0.2237 , loss G :  3.0217 
epochs[2/100]loss D : 0.4889 , loss G :  2.0964 
epochs[3/100]loss D : 0.4374 , loss G :  1.9016 
epochs[4/100]loss D : 0.2808 , loss G :  2.0315 
epochs[5/100]loss D : 0.3598 , loss G :  1.2527 
epochs[6/100]loss D : 0.1787 , loss G :  3.3664 
epochs[7/100]loss D : 0.4384 , loss G :  2.2298 
epochs[8/100]loss D : 0.1949 , loss G :  2.2541 
epochs[9/100]loss D : 0.1814 , loss G :  1.9875 
epochs[10/100]loss D : 0.2593 , loss G :  2.2681 
epochs[11/100]loss D : 0.4520 , loss G :  2.0945 
epochs[12/100]loss D : 0.3799 , loss G :  1.8802 
epochs[13/100]loss D : 0.3664 , loss G :  1.8462 
epochs[14/100]loss D : 0.3056 , loss G :  1.6440 
epochs[15/100]loss D : 0.4337 , loss G :  1.7643 
epochs[16/100]loss D : 0.3630 , loss G :  1.7817 
epochs[17/100]loss D : 0.2470 , loss G :  2.4080 
epochs[18/100]loss D : 0.3741 , loss G :  1.7184 
epochs[19/100]loss D : 0.5323 , loss G :  1.3885 
epochs[20/

# 2. Deep Convolutional Generative Adversarial Network

In [ ]:
import torch
import torch.nn as nn



In [ ]:
# creating Discriminator Class

class Discriminator(nn.Module):

  def __init__(self , channels_img , features_d):
    super(Discriminator , self).__init__()

    self.disc = nn.Sequential(
        nn.Conv2d(
            channels_img , features_d , kernel_size=4 , stride = 2 , padding=1
        ),
        nn.LeakyReLU(0.2),
        self._block(features_d , features_d*2,4,2,1),
        self._block(features_d*2 , features_d*4,4,2,1),
        self._block(features_d*4 , features_d*8,4,2,1),
        nn.Conv2d(features_d*8 ,1,kernel_size=4,stride=2,padding=1 ),
        nn.Sigmoid()
    )

  def _block(self , in_channels , out_channels , kernel_size ,  padding , strides):
    return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size ,
            strides ,
            padding,
            bias=False
        ),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2)
    )

  def forward(self, x):
    return self.disc(x)


# Generator Class

class Generator(nn.Module):

  def __init__(self , z_dims , channels_img ,features_g ):
    super(Generator , self).__init__()
    self.gen= nn.Sequential(
        self._block(z_dims , features_g*16 , 4,2,1),
        self._block(features_g*16 , features_g*8 ,4,2,1),
        self._block(features_g*8 , features_g*4 , 4,2,1),
        self._block(features_g*4 , features_g*2 , 4,2,1),
        nn.ConvTranspose2d(
            features_g*2 , channels_img , kernel_size=4 ,stride = 2 , padding= 1
        ),
        nn.Tanh()
    )

  def _block(self , in_channels , out_channels , kernel_size , stride , padding):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels , out_channels , kernel_size , stride , padding , bias = False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )

  def forward(self , x):
    return self.gen(x)

# Initializing the Weights

In [ ]:
def Initialize_weights(model):
  for m in model.modules():
    if isinstance(m , (nn.Conv2d , nn.ConvTranspose2d , nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data,0.0,0.02)

def test():
  N , in_channels , H , W = 8 , 3 , 64 , 64
  z_dims = 100
  x = torch.randn((N,in_channels , H ,W))
  disc = Discriminator(in_channels , 8)
  Initialize_weights(disc)
  assert disc(x).shape(N,1,1,1)
  gen = Generator(z_dims , in_channels , 8)
  Initialize_weights(gen)
  z = torch.randn((N , z_dims , 1,1))
  assert gen(z).shape(N , in_channels ,H,W )
  print('Success')


# training pipeline



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# hyperparameters

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 2e-4
batch_size = 128
image_size= 64
channel_img = 1
z_dims = 100
num_epochs = 200
features_disc = 64
features_gen = 64

transform = transforms.Compose(
    [transforms.Resize(image_size),
     transforms.ToTensor(),
     transforms.Normalize(
         [0.5 for _ in range(channel_img)] , [0.5 for _ in range(channel_img)]
     )]
)




# data

In [ ]:
dataset = datasets.MNIST(root=",/data",train =True , download=True , transform=transform)
loader = DataLoader(dataset , batch_size= batch_size , shuffle = True)

gen = Generator(z_dims , channel_img , features_gen).to(device)
disc = Discriminator(channel_img , features_disc).to(device)
Initialize_weights(gen)
Initialize_weights(disc)

opt_gen = optim.Adam(gen.parameters() , lr = learning_rate , betas=(0.5 ,0.999))
opt_disc = optim.Adam(disc.parameters() , lr = learning_rate , betas=(0.5 ,0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32,z_dims,1,1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0



for epoch in range(num_epochs):
  for batch_idx ,(real_img , _) in enumerate(loader):
    real_img = real_img.to(device)
    fake_img = gen(fixed_noise)
    noise = torch.randn((batch_size , z_dims , 1,1)).to(device)


    # training discriminator
    disc_real = disc(real_img).reshape(-1)
    loss_disc_real = criterion(disc_real , torch.ones_like(disc_real))
    disc_fake = disc(fake_img).reshape(-1)
    loss_disc_fake = criterion(disc_fake , torch.zeros_like(disc_fake))
    loss_disc = (loss_disc_fake+loss_disc_real)/2
    disc.zero_grad()
    loss_disc.backward(retain_graph=True)
    opt_disc.step()

    # training generator

    output= disc(fake_img).reshape(-1)
    loss_gen = criterion(output , torch.ones_like(output))
    gen.zero_grad()
    loss_gen.backward()
    opt_gen.step()


    with torch.no_grad():
      fake_img = gen(fixed_noise)

      img_grid_real = torchvision.utils.make_grid(
          real_img[:32] , normalize=True
      )
      img_grid_fake = torchvision.utils.make_grid(
          fake_img[:32] , normalize=True
      )

      writer_real.add_image("Real",img_grid_real , global_step=step)
      writer_fake.add_image("fake",img_grid_fake, global_step=step)
    step +=1

# 3. Cycle Generative Adversarial Network

# Disriminator

In [ ]:
class _Block(nn.Module):

  def __init__(self,in_channels , out_channels , stride):
    super().__init__()

    self.conv = nn.Sequential(
        nn.Conv2d(in_channels , out_channels , 4 , stride , padding = 1 , padding_mode="reflect" ),
        nn.InstanceNorm2d(out_channels),
        nn.LeakyReLU(0.2)
    )

  def forward(self , x):
    return self.conv(x)


class CGDiscriminator(nn.Module):

  def __init__(self , in_channels , features = [64,128,256,512]):
    super().__init__()
    self.initial = nn.Sequential(
        nn.Conv2d(
            in_channels,
            features[0],
            kernel_size= 4,
            stride=2,
            padding =1
        ),
        nn.LeakyReLU(0.2)
    )

    layers = []
    in_channels = features[0]
    for feature in features[1:]:
      layers.append(_Block(in_channels , feature , stride =1 if feature == features[-1] else 2))
      in_channels = feature
    layers.append(nn.conv2d(in_channels , 1 , kernel_size =4 , stride =1 , padding =1 , padding_mode = 'reflect'  ))
    self.model = nn.Sequential(*layers)

  def forward(self ,x):
    x = self.initial(x)
    return torch.sigmoid(self.model(x))

# Generator

In [ ]:
class ConvBlock(nn.Module):
  def __init__(self,in_channels , out_channels , down = True , use_act = False , **kwargs ) -> None:
    super().__init__()